In [18]:
import numpy as np
import pandas as pd 

## Loading Dataset

In [19]:
dataset = pd.read_csv('train.csv')
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Data Preprocessing

In [20]:
dataset.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [21]:
# Cabin, PassengerId, Name, Ticket is an irrelevent feature
dataset = dataset.drop(['Cabin', 'PassengerId', 'Name', 'Ticket'], axis=1)

In [23]:
dataset.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


### Encoding the catagorical colums from stings to int

In [24]:
from sklearn import preprocessing

In [26]:
from sklearn.preprocessing import LabelEncoder

In [28]:
label_encoder = LabelEncoder()
dataset['Sex'] = label_encoder.fit_transform(dataset['Sex'])
dataset['Embarked'] = label_encoder.fit_transform(dataset['Embarked'])

In [29]:
dataset.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


## Making dependent variable and independent variable (features)

In [30]:
# DV
Y = dataset['Survived']

In [33]:
Y

0      0
1      1
2      1
3      1
4      0
      ..
884    0
885    1
886    0
887    1
888    0
Name: Survived, Length: 889, dtype: int64

In [32]:
# IDVs
X = dataset.drop(['Survived'], axis=1)

In [34]:
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,2
1,1,0,38.0,1,0,71.2833,0
2,3,0,26.0,0,0,7.9250,2
3,1,0,35.0,1,0,53.1000,2
4,3,1,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...
884,2,1,27.0,0,0,13.0000,2
885,1,0,19.0,0,0,30.0000,2
886,3,0,45.0,1,2,23.4500,2
887,1,1,26.0,0,0,30.0000,0


## Random Forest

In [35]:
from sklearn.ensemble import RandomForestClassifier

In [37]:
rf_model = RandomForestClassifier(n_estimators=1000, max_features=2, oob_score=True)

In [39]:
features = list(X.columns)
features

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

In [40]:
rf_model.fit(X=X, y=Y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [41]:
print("OOB Accuracy : ")
print(rf_model.oob_score_)

OOB Accuracy : 
0.8076490438695163


In [42]:
for feature, imp in zip(features, rf_model.feature_importances_):
    print(feature, " ", imp)

Pclass   0.08578089783361792
Sex   0.2621277133718815
Age   0.25770699935201324
SibSp   0.049361273151496306
Parch   0.04082519962745799
Fare   0.26865159900253893
Embarked   0.03554631766099404


- Sex, Age, Fare are some important features according to the above features importance from the RandomForestClassifer
- max_dept = 3(imp features) * 2 = 6

## Decision Tree

In [43]:
from sklearn import tree

In [44]:
tree_model = tree.DecisionTreeClassifier(max_depth=6, max_leaf_nodes=10)

In [47]:
predictors = pd.DataFrame([dataset['Sex'], dataset['Age'], dataset['Fare']]).T
predictors.head()

,Sex,Age,Fare
0,1.0,22.0,7.2500
1,0.0,38.0,71.2833
2,0.0,26.0,7.9250
3,0.0,35.0,53.1000
4,1.0,35.0,8.0500


In [48]:
tree_model.fit(X=predictors, y=dataset['Survived'])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=6, max_features=None, max_leaf_nodes=10,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [49]:
with open('Dtree2.dot', 'w') as f:
    f = tree.export_graphviz(tree_model, feature_names=['Sex', 'Age', 'Fare'], out_file=f)

<img src="dtree2.png" width=600 height=600 />

In [50]:
tree_model.score(X=predictors, y=dataset['Survived'])

0.8020247469066367